In [ ]:
# Install/Import required packages (uncomment install lines if needed in Colab)
import os
import math
import shutil
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, classification_report

print('TensorFlow version:', tf.__version__)
AUTOTUNE = tf.data.AUTOTUNE

# If running in Colab and you need to install a different TF, uncomment below (use with caution):
# !pip install -q 'tensorflow==2.14.0'

## 1) Prepare dataset: upload, mount or point to local directory

This cell provides two options:
- Option A (recommended in Colab): Mount Google Drive and set `DATA_ROOT` to the folder containing `Train data` and `Test_Data`.
- Option B: Upload a ZIP file directly to the Colab session and it will be extracted to `/content/dataset`.

If you're running locally (not Colab), set `DATA_ROOT` to your dataset root path.

In [ ]:
# Adjust these paths according to your environment.
# If using Colab and Google Drive:
USE_GOOGLE_DRIVE = False  # set to True if mounting Drive in Colab
DRIVE_DATA_PATH = '/content/drive/MyDrive/AniLink_dataset'  # change to your Drive path

# If uploading a ZIP in Colab, set USE_UPLOAD = True to use the upload cell below.
USE_UPLOAD = False

# Default local path fallback (if running locally and your folders are on your machine):
LOCAL_DATA_PATH = r'C:\Users\USER\Desktop\AniLink_project\AniLink'  # adjust if running locally

# Final DATA_ROOT will be determined below
DATA_ROOT = None

if USE_GOOGLE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_ROOT = DRIVE_DATA_PATH

if USE_UPLOAD:
    # This will prompt you to upload a zip file in Colab. Uncomment if needed.
    from google.colab import files
    uploaded = files.upload()
    # find first uploaded zip
    for name in uploaded.keys():
        if name.endswith('.zip'):
            zip_path = name
            break
    else:
        raise RuntimeError('Upload a zip archive containing Train data/ and Test_Data/ folders')
    import zipfile
    extract_dir = '/content/dataset'
    os.makedirs(extract_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_dir)
    DATA_ROOT = extract_dir

if DATA_ROOT is None:
    # prefer the local workspace path if it exists
    if os.path.exists(LOCAL_DATA_PATH):
        DATA_ROOT = LOCAL_DATA_PATH
    else:
        # fallback: expect user to upload or set DATA_ROOT manually
        raise FileNotFoundError('Dataset root not found. Set USE_GOOGLE_DRIVE=True (in Colab) or adjust LOCAL_DATA_PATH to point to your dataset root.')

print('DATA_ROOT set to:', DATA_ROOT)

# Expecting these folders inside DATA_ROOT: 'Train data' and 'Test_Data' (case-sensitive)
TRAIN_DIR = os.path.join(DATA_ROOT, 'Train data')
TEST_DIR = os.path.join(DATA_ROOT, 'Test_Data')
assert os.path.isdir(TRAIN_DIR), f"Train directory not found: {TRAIN_DIR}"
assert os.path.isdir(TEST_DIR), f"Test directory not found: {TEST_DIR}"
print('Train dir classes:', os.listdir(TRAIN_DIR))
print('Test dir classes:', os.listdir(TEST_DIR))

## 2) Create ImageDataGenerators with heavy augmentation and automatic 80/20 split

We use `ImageDataGenerator` which supports `validation_split`. Heavy augmentation includes rotation, flips, shear, zoom, shifts, and brightness.

In [ ]:
# Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123

# Heavy augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.25,
    zoom_range=0.2,
    brightness_range=[0.7,1.3],
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='reflect',
    validation_split=0.2
)

# For validation we only rescale
valid_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# For test set we only rescale
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow from directory with 80/20 split
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    seed=SEED
)

val_generator = valid_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    seed=SEED,
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

num_train = train_generator.samples
num_val = val_generator.samples
num_test = test_generator.samples
print('Train samples:', num_train, 'Val samples:', num_val, 'Test samples:', num_test)

## 3) Build model: MobileNetV3Small (ImageNet) with transfer learning

We freeze the base model, train the head, then optionally unfreeze and fine-tune a few layers.

In [ ]:
from tensorflow.keras.applications import MobileNetV3Small

base_model = MobileNetV3Small(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), include_top=False, weights='imagenet')
base_model.trainable = False  # freeze for initial training

inputs = keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = inputs
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

## 4) Train with callbacks (EarlyStopping, ReduceLROnPlateau) and checkpointing

In [ ]:
EPOCHS = 20

checkpoint_path = 'best_model.h5'
callbacks = [
    keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
]

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks
)

### Optional: Unfreeze and fine-tune
Unfreeze top layers of the base model and continue training with a smaller learning rate to potentially boost accuracy.

In [ ]:
# Fine-tune: unfreeze some layers
base_model.trainable = True

# Freeze bottom layers, unfreeze top n
fine_tune_at = len(base_model.layers) - 20  # tweak as needed
for i, layer in enumerate(base_model.layers):
    layer.trainable = (i >= fine_tune_at)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=5e-6),
              loss='binary_crossentropy',
              metrics=['accuracy'])

FT_EPOCHS = 10
callbacks_ft = [
    keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-8, verbose=1)
]

history_ft = model.fit(
    train_generator,
    epochs=FT_EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks_ft
)

# Merge history objects
def merge_histories(h1, h2):
    for k in h2.history:
        h1.history.setdefault(k, []).extend(h2.history[k])
    return h1

history = merge_histories(history, history_ft)

## 5) Evaluate: accuracy/loss curves, confusion matrix, classification report

In [ ]:
# Load best model weights
model.load_weights(checkpoint_path)

# Plot training curves
acc = history.history.get('accuracy', [])
val_acc = history.history.get('val_accuracy', [])
loss = history.history.get('loss', [])
val_loss = history.history.get('val_loss', [])
epochs_range = range(1, len(acc) + 1)

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Train Acc')
plt.plot(epochs_range, val_acc, label='Val Acc')
plt.legend(); plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.legend(); plt.title('Loss')
plt.show()

# Predict on test set
test_steps = math.ceil(num_test / BATCH_SIZE)
preds = model.predict(test_generator, steps=test_steps, verbose=1)
y_pred = (preds.ravel() >= 0.5).astype(int)
y_true = test_generator.classes[:len(y_pred)]

# Confusion matrix and classification report
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

print('Classification Report:')
target_names = ['healthy (0)', 'FMD (1)']
print(classification_report(y_true, y_pred, target_names=target_names))

# Overall accuracy
acc_test = np.mean(y_pred == y_true)
print(f'Test accuracy: {acc_test*100:.2f}%')

## 6) Convert to TensorFlow Lite with quantization (float16) and check size
Float16 quantization generally reduces model size significantly and is widely compatible on modern devices. If you need integer quantization for full integer support, a representative dataset generator is required (we include an example commented section).

In [ ]:
tflite_model_path = 'model_float16.tflite'

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()

with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

size_mb = os.path.getsize(tflite_model_path) / (1024*1024)
print(f'TFLite model saved to {tflite_model_path} ({size_mb:.2f} MB)')

if size_mb > 6.0:
    print('Warning: model size > 6 MB. Consider using MobileNetV3Small with more aggressive quantization (int8) or pruning.)')

### (Optional) Full integer quantization with representative dataset
Uncomment and use a representative generator if you need int8 quantization. This may produce a smaller model but requires calibration data and may need ops support on target device.

In [ ]:
# Example: representative dataset generator (uncomment to use)
###
### def representative_data_gen():
###     for i in range(100):
###         img, _ = next(train_generator)
###         # img: batch of images
###         for j in range(img.shape[0]):
###             yield [img[j:j+1].astype(np.float32)]
###
### converter = tf.lite.TFLiteConverter.from_keras_model(model)
### converter.optimizations = [tf.lite.Optimize.DEFAULT]
### converter.representative_dataset = representative_data_gen
### converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
### converter.inference_input_type = tf.uint8
### converter.inference_output_type = tf.uint8
### tflite_quant_path = 'model_int8.tflite'
### tflite_quant_model = converter.convert()
### with open(tflite_quant_path, 'wb') as f:
###     f.write(tflite_quant_model)
### print('Saved int8 model:', tflite_quant_path, os.path.getsize(tflite_quant_path)/(1024*1024), 'MB')

## 7) TFLite inference on a single image
This cell shows how to load the `.tflite` file and run inference on one image. Replace `sample_image_path` with your image path.

In [ ]:
import PIL
from PIL import Image

def load_image_for_model(path, target_size=IMG_SIZE):
    img = Image.open(path).convert('RGB')
    img = img.resize(target_size)
    arr = np.array(img).astype('float32') / 255.0
    return arr

# Load tflite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def tflite_predict(image_path):
    img = load_image_for_model(image_path)
    inp = np.expand_dims(img, axis=0).astype(np.float32)
    # If the model is float16 quantized, input is float32; conversion is handled by interpreter
    interpreter.set_tensor(input_details[0]['index'], inp)
    interpreter.invoke()
    out = interpreter.get_tensor(output_details[0]['index'])
    prob = float(out.ravel()[0])
    pred = 1 if prob >= 0.5 else 0
    return pred, prob

# Example usage (uncomment and set a valid image path):
# sample_image_path = '/content/sample.jpg'
# pred, prob = tflite_predict(sample_image_path)
# print('Predicted class:', pred, 'probability:', prob)

## 8) Save Keras model and provide quick local usage notes

We also save the Keras model (`best_model.h5`) for inspection and possible re-conversion locally.

In [ ]:
# Save Keras model
keras_model_path = 'best_model.h5'
model.save(keras_model_path)
print('Saved Keras model to', keras_model_path, 'size (MB):', os.path.getsize(keras_model_path)/(1024*1024))

print('\nDone. Summary:')
print('-', 'Train samples:', num_train)
print('-', 'Validation samples:', num_val)
print('-', 'Test samples:', num_test)
print('-', 'TFLite model:', tflite_model_path, f'({size_mb:.2f} MB)')

**Notes and tips to reach >94% accuracy:**
- Ensure quality of dataset (clear lesion images, consistent labeling). Clean mislabeled samples.
- If classes are imbalanced, consider `class_weight` in `model.fit` or oversampling augmentations for minority class.
- Increase `IMG_SIZE` (e.g., 260 or 288) if lesions are small, but TFLite size may grow.
- Consider training longer or unfreezing more layers during fine-tuning.
- If TFLite size is still >6MB, try int8 quantization with a representative dataset (see commented code).